## Import 1: Classes & Path

In [1]:
# Import 1. 
# Import functions.py into notebook from src folder. 
import sys

# Path relative to your notebook.
sys.path.append("/Users/joel/Desktop/daimil10/CAPSTONE_2/Niger-Political-Shift-Examination/src")

# Import my .py classes.
import data_clean
import plots

# Auto reload.
%load_ext autoreload
%autoreload 2

## Import 2: All Libs

In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style

## Read File & Inital Drop of Columns

In [3]:
# Read file in.
# Dropping columns to trim data to 1996-2022.
# Use clean class and init a cleaner object.
cleaner = data_clean.Clean()

niger = cleaner.read_file('/Users/joel/Desktop/daimil10/CAPSTONE_2/Niger-Political-Shift-Examination/data/Niger.csv')
niger = niger.drop(['Country Name', 'Country Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995'], axis=1)

# Make a metadata df for indicator name, source note, and source_org just in case.
niger_meta = cleaner.read_file('/Users/joel/Desktop/daimil10/CAPSTONE_2/Niger-Political-Shift-Examination/data/Metadata_Indicator_Descriptors.csv')

niger.head()

,Indicator Name,Indicator Code,1996,1997,1998,1999,2000,2001,2002,2003,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Consumption of iodized salt (% of households),SN.ITK.SALT.ZS,NaN,NaN,64.8,NaN,43.4,NaN,NaN,NaN,...,NaN,59.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"International migrant stock, total",SM.POP.TOTL,NaN,NaN,NaN,NaN,122260.0,NaN,NaN,NaN,...,NaN,NaN,189255.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Children in employment, wage workers (% of chi...",SL.WAG.0714.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,NaN,NaN,NaN,NaN,NaN,1.47,NaN,NaN,...,NaN,0.52,NaN,NaN,7.77,NaN,NaN,NaN,NaN,NaN
4,"Unemployment, female (% of female labor force)...",SL.UEM.TOTL.FE.NE.ZS,NaN,NaN,NaN,NaN,NaN,0.93,NaN,NaN,...,NaN,0.43,NaN,NaN,7.79,NaN,NaN,NaN,NaN,NaN


## Describing the Dataset

In [4]:
# Describing the Dataset 1.
niger.describe()

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
count,7.220000e+02,7.010000e+02,7.540000e+02,7.130000e+02,8.840000e+02,8.540000e+02,8.670000e+02,8.900000e+02,9.020000e+02,9.760000e+02,...,9.590000e+02,1.061000e+03,9.910000e+02,9.730000e+02,1.016000e+03,9.540000e+02,8.960000e+02,7.930000e+02,6.650000e+02,2.880000e+02
mean,4.549497e+10,4.881780e+10,5.089087e+10,5.337492e+10,4.356152e+10,4.980169e+10,5.187588e+10,5.175458e+10,5.175860e+10,5.363362e+10,...,1.100600e+11,1.063603e+11,1.197563e+11,1.277102e+11,1.290879e+11,1.496506e+11,1.697309e+11,1.853402e+11,2.441178e+11,4.816017e+11
std,2.594059e+11,2.735129e+11,2.920202e+11,3.012381e+11,2.709639e+11,3.011216e+11,3.167104e+11,3.199028e+11,3.223407e+11,3.402248e+11,...,6.449649e+11,6.538496e+11,7.207883e+11,7.683373e+11,7.966196e+11,8.943148e+11,9.780380e+11,1.039727e+12,1.220390e+12,1.804194e+12
min,-7.320000e+10,-8.152500e+10,-1.104030e+11,-9.471500e+10,-9.605800e+10,-1.094360e+11,-1.350410e+11,-1.345150e+11,-1.675050e+11,-2.201340e+11,...,-6.230320e+11,-7.428600e+11,-9.269520e+11,-7.579830e+11,-8.423970e+11,-1.029010e+12,-1.125410e+12,-1.225480e+12,-1.401350e+12,-1.625610e+12
25%,4.918478e+00,6.021677e+00,5.980128e+00,5.915937e+00,4.294313e+00,6.414036e+00,4.932095e+00,5.342783e+00,5.166796e+00,5.546792e+00,...,5.884318e+00,5.573220e+00,5.478809e+00,5.487638e+00,7.000000e+00,5.300038e+00,5.765524e+00,4.000000e+00,6.000000e+00,3.958333e+00
50%,4.835213e+01,6.200000e+01,4.755490e+01,5.771646e+01,4.008206e+01,4.532787e+01,4.130000e+01,4.109776e+01,3.994440e+01,3.907722e+01,...,4.454976e+01,3.981088e+01,4.173002e+01,4.205525e+01,3.746830e+01,4.040000e+01,4.490927e+01,4.174269e+01,5.473000e+01,3.377863e+01
75%,2.233919e+06,2.677496e+06,1.444991e+06,2.450000e+06,5.000000e+04,1.438667e+05,6.377050e+04,2.514373e+04,3.661700e+04,7.215124e+03,...,4.510000e+04,1.169999e+03,1.008817e+04,2.383697e+04,7.813750e+03,3.130450e+04,3.218892e+05,7.038785e+05,6.799690e+06,6.347849e+06
max,2.650270e+12,2.715640e+12,2.938890e+12,2.921010e+12,2.890930e+12,3.133580e+12,3.308750e+12,3.370440e+12,3.408670e+12,3.656930e+12,...,5.829080e+12,6.252250e+12,6.652170e+12,6.909400e+12,7.337100e+12,8.143540e+12,8.693310e+12,9.136430e+12,9.672180e+12,1.033930e+13


In [5]:
# Describing the Dataset 2.
niger.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1478 entries, 0 to 1477
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Indicator Name  1478 non-null   object 
 1   Indicator Code  1478 non-null   object 
 2   1996            722 non-null    float64
 3   1997            701 non-null    float64
 4   1998            754 non-null    float64
 5   1999            713 non-null    float64
 6   2000            884 non-null    float64
 7   2001            854 non-null    float64
 8   2002            867 non-null    float64
 9   2003            890 non-null    float64
 10  2004            902 non-null    float64
 11  2005            976 non-null    float64
 12  2006            1016 non-null   float64
 13  2007            978 non-null    float64
 14  2008            962 non-null    float64
 15  2009            1005 non-null   float64
 16  2010            1006 non-null   float64
 17  2011            1043 non-null   f

In [6]:
# Describing the Dataset 3.
niger.index

RangeIndex(start=0, stop=1478, step=1)

In [7]:
# Describing the Dataset 4.
niger.columns

Index(['Indicator Name', 'Indicator Code', '1996', '1997', '1998', '1999',
       '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020', '2021', '2022'],
      dtype='object')

In [8]:
# Describing the Dataset 5.
missing_v = niger.isnull().sum()
missing_v

Indicator Name       0
Indicator Code       0
1996               756
1997               777
1998               724
1999               765
2000               594
2001               624
2002               611
2003               588
2004               576
2005               502
2006               462
2007               500
2008               516
2009               473
2010               472
2011               435
2012               400
2013               519
2014               417
2015               487
2016               505
2017               462
2018               524
2019               582
2020               685
2021               813
2022              1190
dtype: int64

In [9]:
# Describing the Dataset 6.
# Using the function that spits outdict of col names and dtypes. 
cleaner.get_column_features(niger)

{'Indicator Name': dtype('O'),
 'Indicator Code': dtype('O'),
 '1996': dtype('float64'),
 '1997': dtype('float64'),
 '1998': dtype('float64'),
 '1999': dtype('float64'),
 '2000': dtype('float64'),
 '2001': dtype('float64'),
 '2002': dtype('float64'),
 '2003': dtype('float64'),
 '2004': dtype('float64'),
 '2005': dtype('float64'),
 '2006': dtype('float64'),
 '2007': dtype('float64'),
 '2008': dtype('float64'),
 '2009': dtype('float64'),
 '2010': dtype('float64'),
 '2011': dtype('float64'),
 '2012': dtype('float64'),
 '2013': dtype('float64'),
 '2014': dtype('float64'),
 '2015': dtype('float64'),
 '2016': dtype('float64'),
 '2017': dtype('float64'),
 '2018': dtype('float64'),
 '2019': dtype('float64'),
 '2020': dtype('float64'),
 '2021': dtype('float64'),
 '2022': dtype('float64')}

## Project Goal #1:
Examine overall Political Stability Index from 1990 to 2022 (32 year span).

## Project Goal #2:
Explore Civil Unrest (Indicators and Incidents) over specified time frame.

## Project Goal #3:
Highlight Political Regime Metrics (multiple coups and multiple leaders). 

## Project Goal #4:
Uncover data points related to corruption perception index & economic indicators. 


## Project Goal #5:
Hypo test (1, 2, 3, 4, or 5)

## Project Goal #6:
Linear Model (1, 2, or 3)